# <a id='toc1_'></a>[Combining data sources](#toc0_)
This notebook pulls cleaned data from the clearning notbooks and combines them into a single dataset for analysis.

**Table of contents**<a id='toc0_'></a>    
- [Combining data sources](#toc1_)    
- [Load libraries](#toc2_)    
- [Load data](#toc3_)    
    - [Add industry data for DK](#toc3_1_1_)    
    - [Add financial data](#toc3_1_2_)    
    - [Add variables calculated on daily data](#toc3_1_3_)    
  - [Add risk-free rate](#toc3_2_)    
- [Feature engineering](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc2_'></a>[Load libraries](#toc0_)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import t as student_t

from pathlib import Path
import sys
parent_dir = Path().resolve().parent
sys.path.insert(0, str(parent_dir))
from libs.functions import *


plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 4), 'figure.dpi': 300})

%load_ext autoreload
%autoreload 2

# <a id='toc3_'></a>[Load data](#toc0_)

In [ ]:
trade = pd.read_csv('data/trade.csv', header=[0,1], index_col=0)
stocks = pd.read_csv('data/stocks.csv')
dk_industry = pd.read_csv('data/dk_industry.csv')
financials = pd.read_csv('data/financials.csv')
rf_rate = pd.read_csv('data/rf_rate.csv')
trade_daily = pd.read_csv('data/trade_daily.csv')

In [3]:
# offsets
quarterly_offset = 3 # quarterly data is 2 months behind end of quarter
annual_offset = 6 # annual data is 6 months behind publication

In [4]:
# convert the index to datetime (the index holds the dates)
trade.index = pd.to_datetime(trade.index)
trade = trade.stack(level=0).reset_index()


# rename columns to have a proper
trade.rename(columns={'level_1': 'ticker'}, inplace=True)

# Data Cleaning and Sorting
trade.drop_duplicates(inplace=True)
trade.dropna(inplace=True) # happens if there was one data point the first day of a given ticker but not the rest of the values (e.g. trade values but no ask or bid)
trade.sort_values(['ticker', 'timestamp'], inplace=True)
trade.reset_index(drop=True, inplace=True)



/var/folders/yw/jsw5n53s1cb1s2q6tt0msrm00000gn/T/ipykernel_60565/3266452105.py:3: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  trade = trade.stack(level=0).reset_index()


In [ ]:
df = pd.merge(trade, stocks[['ticker','shares','NACE']], how='left', on=['ticker'])
# display(df)


### <a id='toc3_1_1_'></a>[Add industry data for DK](#toc0_)

In [ ]:
dk_industry['timestamp'] = pd.to_datetime(dk_industry['timestamp'])
# join stocks and dk_industry on 'NACE industry' 
industry = stocks[['ticker','NACE']].merge(dk_industry, how='left', on='NACE')

# adjust the timestamp to be 2 months behind
industry['timestamp'] = industry['timestamp'] + pd.DateOffset(months=quarterly_offset)
industry['timestamp'] = (
    industry['timestamp']
      .dt.to_period('M')
      .dt.to_timestamp('M')
)

industry = industry.drop(columns=['NACE'])

In [ ]:
df = pd.merge(df, industry, how='left', on=['timestamp', 'ticker'])

#ffil the industry values
for col in industry.columns[2:]:
    df[col] = df[col].groupby(df['ticker']).ffill()


### <a id='toc3_1_2_'></a>[Add financial data](#toc0_)

In [ ]:
financials['timestamp'] = pd.to_datetime(financials['timestamp'])

financials['timestamp'] = financials['timestamp'] + pd.DateOffset(months=annual_offset)
financials['timestamp'] = (
    financials['timestamp']
      .dt.to_period('M')
      .dt.to_timestamp('M')
)

# expand the dataset
financials = (
    financials
        # sort data and find the next timestamp
      .sort_values(['ticker','timestamp'])
      .assign(
        next_fye   = lambda df: df.groupby('ticker')['timestamp'].shift(-1),
        plus_12m   = lambda df: df['timestamp'] + pd.DateOffset(months=12),
        period_end = lambda df: pd.to_datetime(np.where(
                          (df.next_fye - df.timestamp).abs()
                            < 
                          (df.plus_12m   - df.timestamp).abs(),
                          df.next_fye,
                          df.plus_12m
                        ))
      )
      # expand the data
      .assign(timestamp = lambda df: df.apply(expand_monthly, axis=1))
      .explode('timestamp')
      .drop(columns=['next_fye','plus_12m','period_end'])
      .reset_index(drop=True)
)

# display(financials)

In [ ]:
# join df and financials
df = pd.merge(df, financials, how='left', on=['timestamp', 'ticker'])

### <a id='toc3_1_3_'></a>[Add variables calculated on daily data](#toc0_)

In [ ]:
trade_daily['timestamp'] = pd.to_datetime(trade_daily['timestamp'])

In [11]:
df.drop(columns=['ask','bid','adjclose','high','low','open','volume','turnover','div'], inplace=True) # drop overlapping columns

In [ ]:
# join df and beta
df = pd.merge(df, trade_daily, how='left', on=['timestamp', 'ticker'])

## <a id='toc3_2_'></a>[Add risk-free rate](#toc0_)

In [ ]:
rf_rate['timestamp'] = pd.to_datetime(rf_rate['timestamp'])

In [ ]:
# join df and beta
df = pd.merge(df, rf_rate, how='left', on=['timestamp'])

# display(df)

# <a id='toc4_'></a>[Feature engineering](#toc0_)

In [ ]:
# subtract the risk-free rate from the target
df['discount_lag'] = df.groupby('ticker')['discount'].shift(1) # lag the discount rate by 1 month
df['discount_lag'] = (1+df['discount_lag'])**(1/12)-1 # convert to monthly rate
df['target'] = df['target'] - df['discount_lag'] # excess montly return
df.drop(columns=['discount_lag'], inplace=True) # drop the lagged discount rate

df['mcap'] = df['adjclose'] * df['shares'] # market cap

# Green et al features
df['acc'] = (df['netinc'] - df['cashflow'])/df['assets'] # accruals

df['absacc'] = np.abs(df['acc']) # absolute accruals

# df['age'] = (df['timestamp'] - df.groupby('ticker')['timestamp'].transform('min')).dt.days / 365.25 # age of the stock

df['agr'] = (df.groupby('ticker')['assets'].transform(lambda x: x.pct_change(periods=12, fill_method=None))).fillna(0) # yearly asset growth

df['beta'] = df['beta'] # beta

df['betasq'] = df['betasq'] # beta squared

df['bm'] = (df['assets'] - df['liabilities']) / df['mcap'] # book to market ratio #nb to check if debt includes both current and long term liabilities (pretty sure it does)

df['cash'] = (df['cash'] / df['assets']).replace([np.inf, -np.inf], 0) # cash to assets ratio

df['cashdebt'] = (df['cashflow'] / df['debt']).fillna(0).replace([np.inf, -np.inf], 0) # cashflow to debt ratio

df['cashpr'] = ((df['mcap'] + df['longdebt'] - df['assets']) / df['cash']).fillna(0).replace([np.inf, -np.inf], 0)

df['cfp'] = df['cashflow'] / df['mcap'] # cashflow to market cap ratio

df['cfp_ia'] = df['cfp'] - df.groupby(['NACE','timestamp'])['cfp'].transform('mean') # industry adjusted cashflow to market cap ratio

df['chat'] = (df['revenue']/ df['assets']).transform(lambda x: x.pct_change(periods=12, fill_method=None)).fillna(0).replace([np.inf, -np.inf], 0) # change in total assets
df['chatoia'] = df['chat'] - df.groupby(['NACE','timestamp'])['chat'].transform('mean') # industry adjusted change in total assets
df.drop(columns=['chat'], inplace=True) # only used for this calculation

# df['inv'] = df['currentassets'] / df['assets'] # inventory to assets ratio
# df['chinv'] = df.groupby('ticker')['inv'].transform(lambda x: x.pct_change(periods=12, fill_method=None)).fillna(0).replace([np.inf, -np.inf], 0)
# df.drop(columns=['inv'], inplace=True) # only used for this calculation

df['chmom'] = df['mom6m'] - df['mom7_12m'] # change in momentum
df.drop(columns=['mom7_12m'], inplace=True) # only used for this calculation

df['chpm'] = (df['opinc'] / df['revenue']).fillna(0).replace([np.inf, -np.inf], 0) # change in profit margin
df['chpmia'] = df['chpm'] - df.groupby(['NACE','timestamp'])['chpm'].transform('mean') # industry adjusted change in profit margin
df.drop(columns=['chpm'], inplace=True) # only used for this calculation

df['divi'] = df['divi'].fillna(0) # dividens initiation

df['divo'] = df['divo'].fillna(0) # dividens omission

df['dkkvol'] = df['dkkvol'] # dkk volume (dollar volume in Gu et. al.)

df['dy'] = df['div_annual']/df['mcap'] # dividend to price (annualized)

df['egr'] = df.groupby('ticker')['equity'].transform(lambda x: x.pct_change(periods=12, fill_method=None)).fillna(0) # growth in common shareholders equity

df['gma'] = df['grossprofit'] / df['assets'] # gross profitability

df['grltnoa'] = (df['assets'] - df['currentassets']).transform(lambda x: x.pct_change(periods=12, fill_method=None).replace([np.inf, -np.inf], 0)).fillna(0) # growth in long-term net operating assets

df['idiovol'] = df['idiovol'] # idiosyncratic volatility

df['ill'] = df['ill'] # illiquidity (to be calculated on the daily data)

df['indmom'] = df.groupby(['NACE', 'timestamp'])['mom12m'].transform('mean') # industry momentum

df['lev'] = df['debt'] / df['assets'] # leverage

df['lgr'] = df.groupby('ticker')['longdebt'].transform(lambda x: x.pct_change(periods=12, fill_method=None)).fillna(0).replace([np.inf, -np.inf], 0) # long-term debt growth

df['maxret'] = df['maxret'] # maximum daily return

df['mvel1'] = np.log(df['mcap']) # Size (log of market cap)

df['mve_ia'] = df['mvel1'] - df.groupby(['NACE','timestamp'])['mvel1'].transform('mean') # industry adjusted size

# df['nincr'] = None # number of earnings increases

df['operprof'] = df['opinc'] / df['assets'] # operating profitability

# df['orgcap'] = None # organizational capital

# df['pchcapx_ia'] = None # industry adjusted % change in capital expenditures

df['pchcurrat'] = (df.groupby('ticker')['currentassets'].transform(lambda x: x.pct_change(periods=12, fill_method=None)).replace([np.inf, -np.inf], 0)).fillna(0) # % change in current ratio

# df['pchdpr'] = df.groupby('ticker')['depreciation'].transform(lambda x: x.pct_change(periods=12, fill_method=None)).fillna(0) # % change in depreciation

df['pchgm_pchsale'] = (df.groupby('ticker')['grossprofit'].transform(lambda x: x.pct_change(periods=12, fill_method=None)).replace([np.inf, -np.inf], 0) 
                       - df.groupby('ticker')['revenue'].transform(lambda x: x.pct_change(periods=12, fill_method=None)).replace([np.inf, -np.inf], 0)).fillna(0) # % change in gross margin - % change in sales

df['pchquick'] = (df.groupby('ticker')['quick'].transform(lambda x: x.pct_change(periods=12, fill_method=None))).fillna(0).replace([np.inf, -np.inf], 0) # % change in quick ratio

df['pchsale_pchinvt'] = (df.groupby('ticker')['revenue'].transform(lambda x: x.pct_change(periods=12, fill_method=None)).replace([np.inf, -np.inf], 0) 
                         - df.groupby('ticker')['currentassets'].transform(lambda x: x.pct_change(periods=12, fill_method=None)).replace([np.inf, -np.inf], 0)).fillna(0) # % change in sales - % change in inventory

# df['pchsale_pchrect'] = df.groupby('ticker')['revenue'].transform(lambda x: x.pct_change(periods=12, fill_method=None)).fillna(0) - df.groupby('ticker')['receivables'].transform(lambda x: x.pct_change(periods=12, fill_method=None)).fillna(0) # % change in sales - % change in receivables

# df['pchsale_pchxsga'] = None # % change in sales - % change in SG&A

df['saleinv'] = (df['revenue'] / df['currentassets']).fillna(0).replace([np.inf, -np.inf], 0) # sales to inventory ratio

df['pchsaleinv'] = (df.groupby('ticker')['saleinv'].transform(lambda x: x.pct_change(periods=12, fill_method=None))).fillna(0).replace([np.inf, -np.inf], 0) # % change in sales to inventory ratio

df['pctacc'] = ((df['netinc'] - df['cashflow'])/np.abs(df['netinc'].replace(0,0.01))) # % accruals

df['pricedelay'] = df['pricedelay']  # price delay

# df['ps'] = None # financial statements score

df['quick'] = df['quick'] # quick ratio

df['rd'] = (df.groupby('ticker')['rnd'].transform(lambda x: x.pct_change(periods=12, fill_method=None))).fillna(0).replace([np.inf, -np.inf], 0) # R&D increase

df['rd_mve'] = df['rnd'].fillna(0) / df['mcap'] # R&D to market cap ratio

df['rd_sale'] = (df['rnd'].fillna(0) / df['revenue']).fillna(0).replace([np.inf, -np.inf], 0) # R&D to sales ratio

# df['realestate'] = None # real estate holdings

df['retvol'] = df['retvol'] # return volatility

df['roaq'] = df['netinc'] / df['assets'] # return on assets

# df['roavol'] = None # earning volatility

df['roeq'] = df['netinc'] / df['equity'] # return on equity

df['roic'] = df['netinc'] / (df['liabilities_equity']) # return on invested capital

# df['rsup'] = None # revenue surprise

df['salecash'] = (df['revenue'] / df['cash']).fillna(0).replace([np.inf, -np.inf], 0) # sales to cash ratio 

# df['salerec'] =  df['revenue'] / df['receivables'] # sales to receivables ratio

# df['secured'] = None # secured debt

# df['securedind'] = None # secured debt indicator

df['sgr'] = (df.groupby('ticker')['revenue'].transform(lambda x: x.pct_change(periods=12, fill_method=None))).fillna(0).replace([np.inf, -np.inf], 0) # sales growth

# # excluding sin as it gives multicollinearity
# sin_nace = [11,12,92] # alcohol, tobacco, gambling
# df['sin'] = df['NACE'].isin(sin_nace).astype(int) 

# df['sp'] = df['revenue'].fillna(0) / df['mcap'] # sales to price ratio

df['std_dkkvol'] = df['std_dkkvol']  # standard deviation of dkk volume

df['std_turn'] = df['std_turn']  # standard deviation of turnover

df['turn'] = df['turn']  # turnover

# df['zerotrade'] = df['zerotrade'] # zero trading days (commented out as there is no variance)

# Goyal et al features (macro variables)
df['discount'] = df['discount'] # discount rate # Goyal uses 1 year treasury bill rate

df['tms'] = df['10-year'] - df['discount']  # term spread

# df['bm_macro'] = (df.groupby('timestamp')['assets'].transform('sum') - df.groupby('timestamp')['liabilities'].transform('sum')) / df.groupby('timestamp')['mcap'].transform('sum') # book to market ratio

df['dp'] = np.log(df.groupby('timestamp')['div_annual'].transform('sum')) -  np.log(df.groupby('timestamp')['omxcpi'].transform('mean')) # dividend to price (annualized)

df['ep'] = np.log(df.groupby('timestamp')['revenue'].transform('sum')) - np.log(df.groupby('timestamp')['omxcpi'].transform('mean')) # earnings to price

df['svar'] = df['svar'] # index variance


/opt/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [16]:
# drop columns that are not needed
df.drop(columns=['adjclose','div', 'shares', 'omxcpi', 'mcap',
                 'grossoutput', 'intercons', 'prodind', 'valaddoutput',
                 '10-year', 'div_annual', 'zerotrade',
                 'assets', 'debt', 'opinc', 'netinc', 'rnd', 'cashflow', 'revenue', 'grossprofit', 'currentassets', 'volume',
                 'currentliabilities', 'liabilities_equity', 'assets', 'longdebt', 'receivables', 'depreciation', 'equity','liabilities'
                 ], inplace=True)

# remove empty columns (None from above)
for col in df.columns:
    if df[col].isnull().all():
        df.drop(columns=[col], inplace=True)

# drop if mom12m or target is nan (first 12 months and last month for each ticker)
df.dropna(subset=['mom12m','target'], inplace=True)

# drop if any nan
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(f'Number of rows: {len(df)}')
print(f'Number of columns: {len(df.columns)-4}') # -4 for target, timestamp, ticker and NACE (which is deleted after bools are created)

Number of rows: 45578
Number of columns: 61


In [ ]:
# create dummy variables for the industry
industry_dummies = pd.get_dummies(df['NACE'], prefix='NACE_', drop_first=True)
industry_dummies = industry_dummies.astype(int) # convert to int
df = pd.concat([df, industry_dummies], axis=1)
df.drop(columns=['NACE'], inplace=True)
df = df[df['target'] <= 1.5]

# save the data
df.to_csv('data/data.csv', index=False)

In [ ]:
data = df["target"].values

# descriptive stats
mu,    sigma  = data.mean(),  data.std(ddof=0)
mu_lap        = np.median(data)
b_lap         = np.mean(np.abs(data - mu_lap)) # sigma/np.sqrt(2)
nu            = 1                       
bins = 100

# figure
fig, ax = plt.subplots()

# histogram whose bar-tops are *percent*
sns.histplot(
    data,
    bins=bins,
    stat="percent",          
    element="step",
    edgecolor="black",
    color="C0",
    alpha=0.5,
    ax=ax,
)

# x-grid for the pdf curves
x = np.linspace(data.min(), data.max(), bins)
scale = 100 * (x.max() - x.min()) / bins

# overlays
pdf_gauss = (1/(sigma*np.sqrt(2*np.pi))) * np.exp(-0.5 * ((x - mu)/sigma)**2)
ax.plot(x, scale * pdf_gauss, "C1--", lw=2,
        label=rf"Normal($\mu$={mu:.3f}, $\sigma$={sigma:.3f})")

pdf_lap = (1/(2*b_lap)) * np.exp(-np.abs(x - mu_lap)/b_lap)
ax.plot(x, scale * pdf_lap, "C3:", lw=2,
        label=rf"Laplace($\mu_L$={mu_lap:.3f}, b={b_lap:.3f})")

pdf_t = student_t.pdf(x, df=nu, loc=mu, scale=sigma)
ax.plot(x, scale * pdf_t, "C6-.", lw=2,
        label=rf"Student-$t_{{{nu}}}$($\mu$={mu:.3f}, $\sigma$={sigma:.3f})")

ax.set(
    xlabel="Monthly excess return",
    ylabel="Share of observations (%)",
    xlim=(-0.5, 0.5),
)
ax.legend()
ax.grid(axis="y", alpha=0.5)
fig.tight_layout()
fig.savefig("../figs/target_distribution.png", dpi=300)
# plt.show()
plt.close(fig)


In [ ]:
# drop non-predictor and boolean columns
cols_to_drop = ['ticker', 'timestamp'] + [c for c in df.columns if c.startswith('NACE_')]
# identify boolean columns (e.g., dividend initiation, omission, etc.)
bool_cols = df.select_dtypes(include='bool').columns.tolist()
all_drops = cols_to_drop + bool_cols

data = df.drop(columns=all_drops).copy()

# descriptive stats
desc = data.describe().T.loc[:, ['mean', 'std', 'min', '25%', '50%', '75%', 'max']]
desc = desc.rename(columns={
    'mean': 'Mean',
    'std': 'StdDev',
    'min': 'Min',
    '25%': '25\\%',
    '50%': 'Median',
    '75%': '75\\%',
    'max': 'Max'
}).reset_index().rename(columns={'index': 'Acronym'})

desc.sort_values(by='Acronym', inplace=True)
desc.reset_index(drop=True, inplace=True)

descriptions = {
    # rows 1–13
    'absacc':   'Absolute accruals',                                        
    'acc':      'Working capital accruals',                                 
    'aeavol':   'Abnormal earnings announcement volume',                    
    'age':      '\# years since first LSEG Workspace coverage',                   
    'agr':      'Asset growth',                                             
    'baspread': 'Bid-ask spread',                                           
    'beta':     'Beta',                                                     
    'betasq':   'Beta squared',                                             
    'bm':       'Book-to-market',
    'bm_macro':       'Total markets book-to-market',                                            
    'bm_ia':    'Industry-adjusted book to market',                         
    'cash':     'Cash holdings',                                            
    'cashdebt': 'Cash flow to debt',                                        
    'cashpr':   'Cash productivity',                                        

    # rows 14–31
    'cfp':      'Cash flow to price ratio',                                 
    'cfp_ia':   'Industry-adjusted cash flow to price ratio',               
    'chatoia':  'Industry-adjusted change in asset turnover',               
    'chcsho':   'Change in shares outstanding',                            
    'chempia':  'Industry-adjusted change in employees',                    
    'chinv':    'Change in inventory',                                      
    'chmom':    'Change in 6-month momentum',                               
    'chpmia':   'Industry-adjusted change in profit margin',                
    'chtx':     'Change in tax expense',                                    
    'cinvest':  'Corporate investment',                                     
    'convind':  'Convertible debt indicator',                               
    'currat':   'Current ratio',                                            
    'depr':     'Depreciation / PP&E',                                      
    'divi':     'Dividend initiation',                                      
    'divo':     'Dividend omission',                                        
    'dkkvol':   'DKK trading volume',                                    
    'dy':       'Dividend-to-price ratio',                                  
    'ear':      'Earnings announcement return',                             

    # rows 32–41
    'egr':      'Growth in common shareholder equity',                      
    'ep':       'Diff. in log of total earnings accross companies and log of index price',                                        
    'gma':      'Gross profitability',                                     
    'grCAPX':   'Growth in capital expenditures',                          
    'grltnoa':  'Growth in long-term net operating assets',                
    'herf':     'Industry sales concentration',                            
    'hire':     'Employee growth rate',                                    
    'idiovol':  'Idiosyncratic return volatility',                         
    'ill':      'Illiquidity',                                             
    'indmom':   'Industry momentum',                                       

    # rows 42–52
    'invest':   'Capital expenditures and inventory',                      
    'lev':      'Leverage',                                                
    'lgr':      'Growth in long-term debt',                                
    'maxret':   'Maximum daily return',                                    
    'mom12m':   '12-month momentum',                                       
    'mom1m':    '1-month momentum',                                        
    'mom36m':   '36-month momentum',                                       
    'mom6m':    '6-month momentum',                                                                     
    'mvel1':    'Log of market cap',                                                    
    'mve_ia':   'Industry-adjusted mvel1',                                  

    # rows 53–61
    'nincr':        'Number of earnings increases',                        
    'operprof':     'Operating profitability',                             
    'orgcap':       'Organizational capital',                              
    'pchcapx_ia':   '\% change in capital expenditures (industry-adjusted)',
    'pchcurrat':    '\% change in current ratio',                       
    'pchdepr':      '\% change in depreciation',                        
    'pchgm_pchsale':'\% change in gross margin - \% change in sales',    
    'pchquick':     '\% change in quick ratio',                         
    'pchsale_pchinvt':'\% change in sales - \% change in inventory',     
    'pchsale_pchrect':'\% change in sales - \% change in A/R',           
    'pchsale_pchxsga':'\% change in sales - \% change in SG&A',          
    'pchsaleinv':   '\% change sales-to-inventory',                     

    # rows 65–71
    'pctacc':    'Percent accruals',                                   
    'pricedelay':'Price delay',                                        
    'ps':        'Financial statements score (Piotroski)',             
    'quick':     'Quick ratio',                                        
    'rd':        'R\&D increase',                                       
    'rd_mve':    'R\&D to market capitalization',                       
    'rd_sale':   'R\&D to sales',                                       

    # rows 72–82
    'realestate':'Real estate holdings',                               
    'retvol':    'Return volatility',                                  
    'roaq':      'Return on assets',                                   
    'roavol':    'Earnings volatility',                                
    'roeq':      'Return on equity',                                   
    'roic':      'Return on invested capital',                         
    'rsup':      'Revenue surprise',                                   
    'salecash':  'Sales to cash',                                      
    'saleinv':   'Sales to inventory',                                 
    'salerec':   'Sales to receivables',                               
    'secured':   'Secured debt',                                       
    'securedind':'Secured debt indicator',                             
    'sgr':       'Sales growth',                                       
    'sin':       'Sin stocks',                                         
    'sp':        'Sales to price',                                     

    # rows 87–91 (adapted)
    'std_dkkvol':'Volatility of liquidity (dkk trading volume)',       
    'std_turn':  'Volatility of liquidity (share turnover)',           
    'stdacc':    'Accrual volatility',                                 
    'stdcf':     'Cash flow volatility',                                 
    'tang':      'Debt capacity / firm tangibility',                     

    # not in Table A.6
    'dp':       'Diff. in log of total annualized dividends accross companies and log of index price',
    'discount': "National Banken's discount rate",
    'svar':     'Index variance',
    'target':   'Monthly excess return',
    'tms':      'Term spread',
    'turn':     '3-month rolling avg. of monthly trading volume scaled by number of shares',
}

# then attach to DataFrame:
desc['Description'] = desc['Acronym'].map(descriptions)

# reorder the columns
desc = desc[['Acronym', 'Description', 'Mean', 'StdDev', 'Min', '25\\%', 'Median', '75\\%', 'Max']]

# add 1 to the index
desc.index = desc.index + 1
desc.reset_index(drop=False, inplace=True)
desc['Acronym'] = '\\texttt{' + desc['Acronym'] + '}'

# LaTeX output
latex = desc.to_latex(
    index=False,
    float_format="%.2e",
    column_format="clrrrrrrr",
    escape=False
)

latex = latex.replace("\\begin{tabular}{clrrrrrrr}\n", ""
                    ).replace("\\toprule\n", ""
                    ).replace("index & Acronym & Description & Mean & StdDev & Min & 25\% & Median & 75\% & Max \\\\\n",""
                    ).replace("\\midrule\n", ""
                    ).replace("\\bottomrule\n", ""
                    ).replace("\end{tabular}\n", ""
                    ).replace("_", "\\_"
                    )

pattern = re.compile(r'([0-9]\.[0-9]+)e([+-]?\d+)')
latex = pattern.sub(replace_scientific, latex)
latex = latex[:-3]

with open('../tabs/desc_vars.tex', 'w') as f:
    f.write(latex)
# print(latex)
